In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import nip

import os
import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

DEVICE = 'cuda:0'

In [2]:
# path_to_model = '/home/msst/repo/Quantization/ml/llm/pretrained_models/Llama2-7b-trellis'
# qmodel = AutoModelForCausalLM.from_pretrained(
# 	path_to_model, 
# 	trust_remote_code=True,
#     torch_dtype=torch.float16,
# ).to(DEVICE)

# tokenizer = AutoTokenizer.from_pretrained(path_to_model)

In [3]:
path_to_checkpoints = "/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/trellis/"
chpnt_name = 'T256_L16_V2_K2_cbs10_LowBitSym_qtip_ptq_bs5_calib_act_qat_CE_10steps_1ga_AdamW'

qmodel = qlib.QuantizedLlamaForCausalLM.from_pretrained(
	os.path.join(path_to_checkpoints, chpnt_name),
    torch_dtype=torch.float16,
).to(DEVICE)

model_name = 'Llama2-7b-hf'
tokenizer = qlib.load_tokenizer(model_name)

LOL, Im custom!


In [4]:
dataloader = qlib.QATDataset(
    config=nip.load('/home/msst/repo/Quantization/configs/data/wikitext_test_seqlen4096.yaml'),
    tokenizer=tokenizer
).get_dataloader()

with torch.amp.autocast('cuda', dtype=torch.float16):
    res = qlib.evaluate(qmodel.half(), dataloader, print_times=25)
    print(res)


 10%|▉         | 2/21 [00:29<04:35, 14.50s/it]

103.41623870935301


 10%|▉         | 2/21 [00:31<04:59, 15.74s/it]


KeyboardInterrupt: 

In [6]:
path_to_save = os.path.join(
	path_to_checkpoints, 
	f'{chpnt_name}_calib_act'
)
qmodel.half().save_pretrained(path_to_save)

In [5]:
### 25.07 ppl init
## 7.016 ptq (ns=128, bs=2, ne=1)
## 7.545 ptq (ns=128, bs=2, ne=1, fp acts)


## 7.058 ptq (ns=128, bs=2, ne=2)
# 6.654 ptq + qat (KD+CE, 25 steps)
# 6.598 ptq + qat (CE, 25 steps)
# 6.547 ptq + qat (KD, 10 steps)
# 6.511 ptq + qat (KD, 5 steps)

## 6.779 ptq (ns=320, bs=5, ne=2)
# 6.579 ptq + qat (KD, 2 steps)
# 6.493 ptq + qat (KD, 5 steps) # PiQA 0.7584 # WINO 0.6606 # ArcE 0.7071 # ArcC 0.3712
# 7.372 ptq + qat (KD, 25 steps)
# 7.236 ptq + qat (KD, 10 steps 4 ga Adafactor)
# > 7.5 ptq + qat (CE, 10 steps 4 ga Adafactor)
# 6.567 ptq + qat (CE, 10 steps 4 ga Adam)
# 6.612 ptq + qat (CE, 10 steps 4 ga Adam small lr)

## 6.797 ptq (ns=320, bs=5, ne=4) 

### 39 ppl init
# 6.778 ptq (ns=128, bs=2, ne=2)
# 6.565 ptq + qat (CE, 10 steps)
# 6.478 ptq + qat (KD, 25 steps)
# 6.420 ptq + qat (KD+CE, 25 steps)